# Create 5-degree monthly average gridded dataset for SIF and XCO2 over Cont. US with land grid supplied
---

In [1]:
import sys
sys.path.insert(0, "../src")

import pandas as pd 
import xarray as xr

import data_utils
import krige_tools
from stat_tools import apply_detrend

In [2]:
# collect gridded dataset, standardize, and establish a common domain
df_grid = (
    pd.DataFrame(krige_tools.land_grid(
        res=5, lon_lwr=-125, lon_upr=70, lat_lwr=10, lat_upr=50))
    .rename(columns={0:"lat", 1:"lon"})
    .assign(land=lambda x:1)
    .set_index(["lon", "lat"])
)

def detrend_standardize_da(da):
    da, _ = apply_detrend(da)
    return (da - da.mean(dim="time")) / da.std(dim="time")

with xr.open_dataset("../data/exp_pro/OCO2_5deg_monthly.nc") as ds:
    ds = ds[["sif", "xco2"]]
    ds["sif"] = detrend_standardize_da(ds.sif)
    ds["xco2"] = detrend_standardize_da(ds.xco2)
    # residuals
    ds["sif_res"] = ds.sif - ds.sif.mean(dim="time")
    ds["xco2_res"] = ds.xco2 - ds.xco2.mean(dim="time")

df = ds.to_dataframe().join(df_grid).dropna(subset=["land"]).reset_index()
# df["lat_lon"] = df[["lat", "lon"]].apply(tuple, axis=1)
df

/home/jj829/sif-data-fusion/conda-env/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1665: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/home/jj829/sif-data-fusion/conda-env/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1665: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


,lat,lon,time,sif,xco2,sif_res,xco2_res,land
0,12.5,-92.5,2014-09-01,0.856688,-1.930289,0.856688,-1.930289,1.0
1,12.5,-92.5,2014-10-01,1.054755,-1.655283,1.054755,-1.655283,1.0
2,12.5,-92.5,2014-11-01,-0.277895,-0.597934,-0.277895,-0.597934,1.0
3,12.5,-92.5,2014-12-01,-0.846134,-0.160481,-0.846134,-0.160481,1.0
4,12.5,-92.5,2015-01-01,-1.303775,-0.192856,-1.303775,-0.192856,1.0
...,...,...,...,...,...,...,...,...
15820,47.5,67.5,2020-07-01,-0.028552,-0.939483,-0.028552,-0.939483,1.0
15821,47.5,67.5,2020-08-01,-0.454206,-1.641971,-0.454206,-1.641971,1.0
15822,47.5,67.5,2020-09-01,-0.764465,-1.220648,-0.764465,-1.220648,1.0
15823,47.5,67.5,2020-10-01,-1.051660,-0.633847,-1.051660,-0.633847,1.0


In [4]:
df_conus = df[["lat", "lon", "time", "sif_res", "xco2_res"]]
df_conus.to_csv("../data/exp_pro/OCO2_5deg_monthly_conus.csv", index=False)